<a href="https://colab.research.google.com/github/zachmiller280/cta-api-wrapper/blob/main/cta_api_request.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#### Helper functions

def check_substrings(input_string, substrings= {'^': 'bus', '*': 'train'}):
    """
    Checks if a string contains specific substrings and assigns a value accordingly.

    Args:
        input_string: The string to check.
        substrings: A dictionary where keys are substrings to search for and values
                    are the corresponding assigned values.

    Returns:
        A list of values corresponding to the found substrings.
        Returns an empty list if no substrings are found.
    """
    input_string = str(input_string)

    found_values = []
    for key, value in substrings.items():
        if key in input_string:
            found_values.append(value)
    return ' & '.join(found_values) if found_values else None

Train API Documentation: https://www.transitchicago.com/developers/ttdocs/

Bus API Documentation: https://www.transitchicago.com/developers/bustracker/


https://www.transitchicago.com/diydisplay/?s1=41420&s2=5756&n2=Halsted+%26+Waveland%2fBroadway+Terminal&ps=5756&s3=17390&n3=Halsted+%26+Waveland&s4=12553&n4=Addison+%26+Fremont&size=small&w=0


https://www.transitchicago.com/diydisplay/?s1=41420&s2=5756&n2=Halsted+%26+Waveland%2fBroadway+Terminal&ps=5756&s3=17390&n3=Halsted+%26+Waveland&s4=12553&n4=Addison+%26+Fremont&size=small&w=0

https://www.transitchicago.com/diydisplay/?s1=41420&s2=5756&n2=Halsted+%26+Waveland%2fBroadway+Terminal&ps=5756&s3=17390&n3=Halsted+%26+Waveland&s4=12553&n4=Addison+%26+Fremont&size=small&w=0

In [2]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import urllib.request
from io import StringIO

TRAIN_API_KEY = '01663f35bd23469c83f10526e249f08e'
BUS_API_KEY = 'w7nk49ApKGyDgChZuSdRZUhVB'

my_lat = 41.949084
my_lon = -87.651395

### Getting Stops and Routes Data

This data is not likely to change very often. More info here: https://www.transitchicago.com/developers/gtfs/

`stops.txt` - list of stop locations for bus and train, also includes parent station info for trains

`routes.txt` - route list with unique identifiers


Note:

0-29999                  = Bus stops

30000-39999          = Train stops

40000-49999          = Train stations (parent stops)


### CTA_STOP_XFERS Data

This dataset shows all stops and routes as well as the routes you can transfer to.


**Notes:**

`transfer_to_route_id`: '^' = transfer to bus route , '*' = transfer to train


In [4]:
# URL of the file
url = "https://www.transitchicago.com/downloads/sch_data/CTA_STOP_XFERS.txt"

substrings = {}

# Defining headers for dataframe
headers = ['route_id','route_type', 'stop_name', 'stop_id', 'stop_lat','stop_lon','heading_degrees','transfer_to_route_id']

# Set up a request with a User-Agent header
req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})

# Open the URL and read the content
with urllib.request.urlopen(req) as response:
    data = response.read().decode('utf-8')

# Use StringIO to treat the string as a file-like object for pandas
transfer_df = pd.read_csv(StringIO(data),header=None)
transfer_df.columns = headers

transfer_df['transfer_to_route_type'] = transfer_df['transfer_to_route_id'].apply(lambda x: check_substrings(x))

# Notes: transfer_to_route_id '^' = transfer to bus route , '*' = transfer to train


# Display our dataframe
display(transfer_df)

# Save transfer_df to csv
transfer_df.to_csv('data/CTA_STOP_XFERS.csv',index = False)

,route_id,route_type,stop_name,stop_id,stop_lat,stop_lon,heading_degrees,transfer_to_route_id,transfer_to_route_type
0,152,3,Addison & Halsted,17298,41.947298,-87.649178,90,8-North^;8-South^,bus
1,152,3,Addison & Hamlin,12513,41.946408,-87.722580,84,NaN,None
2,152,3,Addison & Harlem,17358,41.945240,-87.806693,102,90-North^;90-South^,bus
3,152,3,Addison & Hoyne,12530,41.946782,-87.681080,89,NaN,None
4,152,3,Addison & Karlov,12510,41.946313,-87.729287,71,NaN,None
...,...,...,...,...,...,...,...,...,...
14664,X9,1,Ashland Orange Line Station,4164,41.839099,-87.665390,248,31-East^;31-West^;Orange*,bus & train
14665,X9,1,Ashland/63rd Street (Green Line),6179,41.778936,-87.664081,358,63-East^;63-West^;Green*,bus & train
14666,X9,1,Irving Park & Clark,5668,41.954310,-87.662413,90,22-North^;22-South^,bus
14667,X9,1,Irving Park & Fremont,15930,41.954603,-87.651532,302,80-East^;80-West^,bus


OSError: Cannot save file into a non-existent directory: 'data'

In [ ]:
# prompt: Using dataframe transfer_df: create a lookup table using route_id, route_type, and stop_id

# Create a lookup table with route_id, route_type, and stop_id
lookup_table = transfer_df[['route_id', 'route_type', 'stop_id']].drop_duplicates()

# Display the first few rows of the lookup table
print(lookup_table.head())


In [ ]:
# Get the google_transit.zip from https://www.transitchicago.com/downloads/sch_data/ and read stops.txt into a dataframe then save to csv

import pandas as pd
import requests
import zipfile
import io

# Download the zip file
url = "https://www.transitchicago.com/downloads/sch_data/google_transit.zip"
response = requests.get(url)
response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

# Read the zip file in memory
zip_file = zipfile.ZipFile(io.BytesIO(response.content))

# Read stops.txt into a pandas DataFrame
stops_df = pd.read_csv(zip_file.open('stops.txt'))
stops_df['stop_type'] = stops_df['stop_id'].apply(lambda x: 'bus' if x < 30000 else 'train' if x < 40000 else 'parent train stop')


# Read routes.txt into a pandas DataFrame
routes_df = pd.read_csv(zip_file.open('routes.txt'))

# Display the DataFrames
display(stops_df)
display(routes_df)


# Save DataFrames into csv's
stops_df.to_csv('data/stops.csv',index = False)
routes_df.to_csv('data/routes.csv',index = False)

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,stop_type
0,1,1.0,Jackson & Austin Terminal,"Jackson & Austin Terminal, Northeastbound, Bus...",41.876330,-87.774111,0,NaN,1,bus
1,2,2.0,5900 W Jackson,"5900 W Jackson, Eastbound, Southside of the St...",41.877075,-87.771324,0,NaN,1,bus
2,4,4.0,5700 W Jackson,"5700 W Jackson, Eastbound, Southside of the St...",41.876992,-87.768264,0,NaN,1,bus
3,6,6.0,Jackson & Lotus,"Jackson & Lotus, Eastbound, Southeast Corner",41.876521,-87.761452,0,NaN,1,bus
4,7,7.0,5351 W Jackson,"5351 W Jackson, Eastbound, Southside of the St...",41.876560,-87.758931,0,NaN,1,bus
...,...,...,...,...,...,...,...,...,...,...
11154,60009,NaN,Sox-35th (33rd St entry),Auxiliary entrance farecard required,41.834580,-87.630700,2,40190.0,2,parent train stop
11155,60010,NaN,Sox-35th (35th St entry),Main entrance accessible,41.831060,-87.630700,2,40190.0,1,parent train stop
11156,60011,NaN,Wilson (Wilson Av south side entry),Main entrance on south side of Wilson accessib...,41.965280,-87.657900,2,40540.0,1,parent train stop
11157,60012,NaN,Wilson (Sunnyside Av entry),Entrance on Sunnyside west of B'way accessible...,41.963500,-87.657300,2,40540.0,1,parent train stop


,route_id,route_short_name,route_long_name,route_type,route_url,route_color,route_text_color
0,1,1,Bronzeville/Union Station,3,http://www.transitchicago.com/bus/1/,99999C,ffffff
1,2,2,Hyde Park Express,3,http://www.transitchicago.com/bus/2/,b71234,ffffff
2,3,3,King Drive,3,http://www.transitchicago.com/bus/3/,99999C,ffffff
3,4,4,Cottage Grove,3,http://www.transitchicago.com/bus/4/,99999C,ffffff
4,X4,X4,Cottage Grove Express,3,http://www.transitchicago.com/bus/X4/,b71234,ffffff
...,...,...,...,...,...,...,...
126,Blue,NaN,Blue Line,1,http://www.transitchicago.com/blueline/,00A1DE,FFFFFF
127,Pink,NaN,Pink Line,1,http://www.transitchicago.com/pinkline/,E27EA6,FFFFFF
128,G,NaN,Green Line,1,http://www.transitchicago.com/greenline/,009B3A,FFFFFF
129,Org,NaN,Orange Line,1,http://www.transitchicago.com/orangeline/,F9461C,FFFFFF


OSError: Cannot save file into a non-existent directory: 'data'

### Getting Nearest Stop based on Lat / Long

In [ ]:
from geopy.distance import geodesic

def get_nearest_stops(latitude, longitude, max_distance, stop_type=None):
    # Load the stops data
    stops_df = pd.read_csv('data/stops.csv',index_col=False)

    # Create a new column for distance calculation
    stops_df['distance'] = stops_df.apply(lambda row: geodesic((latitude, longitude), (row['stop_lat'], row['stop_lon'])).miles, axis=1)

    # Filter stops within the max_distance and sort by stop type and distance, prioritizing train stops
    filtered_stops = stops_df[stops_df['distance'] <= max_distance].sort_values(by=['stop_type', 'distance'], ascending=[False,True])

    filtered_stops = filtered_stops[filtered_stops.stop_type != 'train']


    return filtered_stops


get_nearest_stops(my_lat, my_lon, 0.25)

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,stop_type,distance
11126,41420,NaN,Addison (Red),NaN,41.947316,-87.653624,1,NaN,1,parent train stop,0.167562
9915,17390,17390.0,Halsted & Waveland,"Halsted & Waveland, Northbound, Northeast Corner",41.949249,-87.649455,0,NaN,1,bus,0.100615
8051,12553,12553.0,Addison & Fremont,"Addison & Fremont, Westbound, Northeast Corner",41.947357,-87.651802,0,NaN,1,bus,0.120985
3794,5756,5756.0,Halsted & Waveland/Broadway Terminal,"Halsted & Waveland/Broadway Terminal, Eastboun...",41.949834,-87.649155,0,NaN,1,bus,0.126451
8907,14904,14904.0,Addison & Fremont,"Addison & Fremont, Eastbound, Southeast Corner",41.947230,-87.651667,0,NaN,1,bus,0.128719
3795,5757,5757.0,Waveland & Broadway,"Waveland & Broadway, Westbound, Northwest Corner",41.949195,-87.648690,0,NaN,1,bus,0.139581
3545,5360,5360.0,Broadway & Waveland,"Broadway & Waveland, Southbound, Northwest Corner",41.949270,-87.648514,0,NaN,1,bus,0.148980
8050,12552,12552.0,Addison & Halsted,"Addison & Halsted, Westbound, Northeast Corner",41.947379,-87.649323,0,NaN,1,bus,0.158893
3570,5405,5405.0,Broadway & Waveland,"Broadway & Waveland, Northbound, Southeast Corner",41.949104,-87.648250,0,NaN,1,bus,0.162024
8905,14901,14901.0,Halsted & Addison,"Halsted & Addison, Southbound, Southwest Corner",41.947144,-87.649578,0,NaN,1,bus,0.163350


In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

STOP_ID = '41420'  # e.g., for a Red Line station

def get_train_arrivals(STOP_ID):
    url = f"http://lapi.transitchicago.com/api/1.0/ttarrivals.aspx?key={TRAIN_API_KEY}&mapid={STOP_ID}"
    response = requests.get(url)
    root = ET.fromstring(response.content)
    arrivals_data = []
    for et in root.findall("eta"):
        arrival_data = {}
        for child in et:
          arrival_data[child.tag] = child.text
        arrivals_data.append(arrival_data)

    arrivals_df = pd.DataFrame(arrivals_data)
    arrivals_df['arrT'] = pd.to_datetime(arrivals_df['arrT'])
    arrivals_df['prdt'] = pd.to_datetime(arrivals_df['prdt'])

    arrivals_df['expected_wait'] = ((arrivals_df['arrT'] - arrivals_df['prdt']).dt.total_seconds() / 60)
    arrivals_df['expected_wait'] = arrivals_df['expected_wait'].apply(lambda x: 'DUE' if x <= 1 else str(int(x)))


    arrivals_df['direction'] = arrivals_df['trDr'].apply(lambda x: 'Northbound' if x == '1' else 'Southbound')

    return arrivals_df

arrivals_df = get_train_arrivals('41420')
arrivals_df


,staId,stpId,staNm,stpDe,rn,rt,destSt,destNm,trDr,prdt,...,isApp,isSch,isDly,isFlt,flags,lat,lon,heading,expected_wait,direction
0,41420,30273,Addison,Service toward Howard,811,Red,30173,Howard,1,2025-05-07 14:44:53,...,1,0,0,0,None,41.9403,-87.65339,359,DUE,Northbound
1,41420,30274,Addison,Service toward 95th/Dan Ryan,912,Red,30089,95th/Dan Ryan,5,2025-05-07 14:44:43,...,0,0,0,0,None,41.99026,-87.65908,177,12,Southbound
2,41420,30273,Addison,Service toward Howard,922,Red,30173,Howard,1,2025-05-07 14:44:53,...,0,0,0,0,None,41.89166,-87.62802,358,15,Northbound
3,41420,30274,Addison,Service toward 95th/Dan Ryan,829,Red,30089,95th/Dan Ryan,5,2025-05-07 14:45:20,...,0,0,0,0,None,41.94743,-87.65363,179,15,Southbound
4,41420,30273,Addison,Service toward Howard,823,Red,30173,Howard,1,2025-05-07 14:45:17,...,0,0,0,0,None,41.87945,-87.62765,357,18,Northbound
5,41420,30273,Addison,Service toward Howard,923,Red,30173,Howard,1,2025-05-07 14:45:12,...,0,0,0,0,None,41.85453,-87.63128,349,24,Northbound
6,41420,30273,Addison,Service toward Howard,915,Red,30173,Howard,1,2025-05-07 14:45:19,...,0,0,0,0,None,41.81605,-87.63027,358,30,Northbound
7,41420,30273,Addison,Service toward Howard,817,Red,30173,Howard,1,2025-05-07 14:45:11,...,0,0,0,0,None,41.76497,-87.62569,359,40,Northbound


## CTA Train API Wrapper Class

In [ ]:
class CTATrainTracker:
  BASE_URL = "https://lapi.transitchicago.com/api/1.0"

  def __init__(self, api_key):
      self.api_key = api_key

  def _get(self, endpoint, params=None):
      if params is None:
          params = {}
      params['key'] = self.api_key
      params['outputType'] = 'json'

      url = f"{self.BASE_URL}/{endpoint}"

      try:
        response = requests.get(url, params=params)
        response.raise_for_status()
      except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e}")
        return None
      except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        return None

      print(endpoint)
      return response.json()

  def get_arrival_predictions(self, mapid=None,stpid=None,max=None,rt=None):
    """
    Returns arrival predictions for a given stop ID.

    Parameters:
        mapid (int): Station ID (required if stpid not specified) - five digits in 4xxxx range - Equivelant to staId
        stpid (int): Stop ID (required if mapid not specified) - five digits in 3xxxx range
        max (int): Maximum results (OPTIONAL - if not specified, all available results for the requested stop or station will be returned)
        rt (str): Route Code (OPTIONAL - if not specified, all available results for the requested stop or station will be returned)
    """
    if not mapid and not stpid:
        raise ValueError("Either 'mapid' or 'stpid' must be provided.")

    params = {}
    if mapid:
        params['mapid'] = str(mapid)
    if stpid:
        params['stpid'] = str(stpid)

    if max:
        params['max'] = max
    if rt:
      params['rt'] = rt

    return self._get("ttarrivals.aspx", params)

  def get_follow_train(self, runnumber):
    """
    Returns a list of arrival predictions for a given train at all subsequent stations for which that train is estimated to arrive, up to 60 minutes in the future or to the end of its trip.

    Parameters:
        runnumber (int): Train run number (required)
    """

    params = {}
    params['runnumber'] = runnumber

    return self._get("ttfollow.aspx", params)

  def get_locations(self, rt):
    """
    Returns a list of in-service trains and basic info and their locations for one or more specified ‘L’ routes.

    Parameters:
        rt (str): Route Code (required)
    """

    params = {}
    params['rt'] = rt

    return self._get("ttpositions.aspx", params)

In [ ]:
# Example usage of the CTATrainTracker class
cta_tracker = CTATrainTracker(api_key=TRAIN_API_KEY)

# Get arrival predictions for a specific station
arrival_predictions = cta_tracker.get_arrival_predictions(mapid=41420)
print(arrival_predictions)

# Get arrival predictions for a specific stop
arrival_predictions = cta_tracker.get_arrival_predictions(stpid=30274)
print(arrival_predictions)


# Get follow train info
follow_train_info = cta_tracker.get_follow_train(runnumber=918)
print(follow_train_info)

# Get train positions for a route
train_positions = cta_tracker.get_locations(rt='Red')
print(train_positions)


ttarrivals.aspx
{'ctatt': {'tmst': '2025-05-07T15:59:49', 'errCd': '0', 'errNm': None, 'eta': [{'staId': '41420', 'stpId': '30274', 'staNm': 'Addison', 'stpDe': 'Service toward 95th/Dan Ryan', 'rn': '915', 'rt': 'Red', 'destSt': '30089', 'destNm': '95th/Dan Ryan', 'trDr': '5', 'prdt': '2025-05-07T15:59:31', 'arrT': '2025-05-07T16:01:31', 'isApp': '0', 'isSch': '0', 'isDly': '0', 'isFlt': '0', 'flags': None, 'lat': '41.94743', 'lon': '-87.65363', 'heading': '179'}, {'staId': '41420', 'stpId': '30273', 'staNm': 'Addison', 'stpDe': 'Service toward Howard', 'rn': '928', 'rt': 'Red', 'destSt': '30173', 'destNm': 'Howard', 'trDr': '1', 'prdt': '2025-05-07T15:59:26', 'arrT': '2025-05-07T16:02:26', 'isApp': '0', 'isSch': '0', 'isDly': '0', 'isFlt': '0', 'flags': None, 'lat': '41.93161', 'lon': '-87.65307', 'heading': '357'}, {'staId': '41420', 'stpId': '30273', 'staNm': 'Addison', 'stpDe': 'Service toward Howard', 'rn': '916', 'rt': 'Red', 'destSt': '30173', 'destNm': 'Howard', 'trDr': '1', 'p

## CTA Bus API Wrapper Class



In [ ]:
import requests

class CTABusTracker:
    BASE_URL = "http://www.ctabustracker.com/bustime/api/v3"

    def __init__(self, api_key):
        self.api_key = api_key

    def _get(self, endpoint, params=None):
        if params is None:
            params = {}
        params['key'] = self.api_key
        params['format'] = 'json'

        url = f"{self.BASE_URL}/{endpoint}"
        response = requests.get(url, params=params)
        response.raise_for_status()
        print(endpoint)
        return response.json()

    def get_time(self):
        """
        Returns the current time from the CTA Bus Tracker system.
        No additional parameters required.
        """
        return self._get("gettime")

    def get_vehicles(self, vid=None, rt=None):
        """
        Returns the locations of vehicles.
        Parameters:
            vid (str): Optional. Comma-separated list of vehicle IDs.
            rt (str): Optional. Comma-separated list of route designators.
        Note: At least one of 'vid' or 'rt' must be provided.
        """
        if not vid and not rt:
            raise ValueError("At least one of 'vid' or 'rt' must be provided.")
        params = {}
        if vid:
            params['vid'] = vid
        if rt:
            params['rt'] = rt
        return self._get("getvehicles", params)

    def get_routes(self):
        """
        Returns a list of all available bus routes.
        No additional parameters required.
        """
        return self._get("getroutes")

    def get_directions(self, rt):
        """
        Returns the directions for a given route.
        Parameters:
            rt (str): Required. Route designator.
        """
        if not rt:
            raise ValueError("Parameter 'rt' is required.")
        params = {'rt': rt}
        return self._get("getdirections", params)

    def get_stops(self, rt, dir, bound=None):
        """
        Returns the stops for a given route and direction.
        Parameters:
            rt (str): Required. Route designator.
            dir (str): Required. Direction (e.g., 'Northbound', 'Southbound').
            bound (str): Optional. Bound (e.g., 'N', 'S', 'E', 'W').
        """
        if not rt or not dir:
            raise ValueError("Parameters 'rt' and 'dir' are required.")
        params = {'rt': rt, 'dir': dir}
        if bound:
            params['bound'] = bound
        return self._get("getstops", params)

    def get_patterns(self, pid=None, rt=None):
        """
        Returns the patterns for a given route or pattern ID.
        Parameters:
            pid (str): Optional. Comma-separated list of pattern IDs.
            rt (str): Optional. Comma-separated list of route designators.
        Note: At least one of 'pid' or 'rt' must be provided.
        """
        if not pid and not rt:
            raise ValueError("At least one of 'pid' or 'rt' must be provided.")
        params = {}
        if pid:
            params['pid'] = pid
        if rt:
            params['rt'] = rt
        return self._get("getpatterns", params)

    def get_predictions(self, stpid=None, rt=None, vid=None, top=None):
        """
        Returns arrival predictions for a given stop ID, route, or vehicle ID.
        Parameters:
            stpid (str): Comma-separated list of stop IDs.
            rt (str): Comma-separated list of route designators.
            vid (str): Optional. Comma-separated list of vehicle IDs.
            top (int): Optional. Maximum number of predictions to return.
        Note: At least one of 'stpid', 'rt', or 'vid' must be provided.
        """
        if not stpid and not rt and not vid:
            raise ValueError("At least one of 'stpid', 'rt', or 'vid' must be provided.")
        params = {}
        if stpid:
            params['stpid'] = stpid
        if rt:
            params['rt'] = rt
        if vid:
            params['vid'] = vid
        if top:
            params['top'] = top
        return self._get("getpredictions", params)

    def get_locales(self, locale=None):
        """
        Returns the list of available locales.
        Parameters:
            locale (str): Optional. Locale code (e.g., 'en', 'es').
        """
        params = {}
        if locale:
            params['locale'] = locale
        return self._get("getlocalelist", params)

    def get_detours(self, rt=None):
        """
        Returns detour information.
        Parameters:
            rt (str): Optional. Comma-separated list of route designators.
        """
        params = {}
        if rt:
            params['rt'] = rt
        return self._get("getdetours", params)

    def get_enhanced_detours(self):
        """
        NOTE: This endpoint is not currently available.
        Returns enhanced detour information.

        """
        return self._get("getenhanceddetours")


In [ ]:
# Replace 'YOUR_API_KEY' with your actual CTA API key
cta = CTABusTracker(api_key=BUS_API_KEY)

# Get current time
current_time = cta.get_time()
print(current_time)

# Get vehicle locations for route 22
vehicles = cta.get_vehicles(rt='22')
print(vehicles)

# Get available routes
routes = cta.get_routes()
print(routes)

# Get directions for route 22
directions = cta.get_directions(rt='22')
print(directions)

# Get stops for route 22 and direction 'Northbound'
stops = cta.get_stops(rt='22', dir='Northbound')
print(stops)

# Get patterns for route 22
patterns = cta.get_patterns(rt='22')
print(patterns)

# Get predictions for stop ID 14787
predictions = cta.get_predictions(stpid='14787')
print(predictions)

# Get available locales
locales = cta.get_locales()
print(locales)

# Get detours for route 22
detours = cta.get_detours(rt='22')
print(detours)

# Get enhanced detours for route 22
enhanced_detours = cta.get_enhanced_detours()
print(enhanced_detours)


gettime
{'bustime-response': {'tm': '20250507 15:00:37'}}
getvehicles
{'bustime-response': {'vehicle': [{'vid': '8774', 'tmstmp': '20250507 15:00', 'lat': '41.87398147583008', 'lon': '-87.63066101074219', 'hdg': '180', 'pid': 3932, 'rt': '22', 'des': 'Howard', 'pdist': 0, 'dly': False, 'tatripid': '1040920', 'origtatripno': '264210381', 'tablockid': '22 -552', 'zone': '', 'mode': 1, 'psgld': 'N/A', 'stst': 54360, 'stsd': '2025-05-07'}, {'vid': '1878', 'tmstmp': '20250507 15:00', 'lat': '41.87283369700114', 'lon': '-87.6291566212972', 'hdg': '354', 'pid': 3932, 'rt': '22', 'des': 'Howard', 'pdist': 1255, 'dly': False, 'tatripid': '1040919', 'origtatripno': '264210380', 'tablockid': '22 -521', 'zone': '', 'mode': 1, 'psgld': 'N/A', 'stst': 53820, 'stsd': '2025-05-07'}, {'vid': '8846', 'tmstmp': '20250507 15:00', 'lat': '41.895336866378784', 'lon': '-87.62973141670227', 'hdg': '1', 'pid': 3932, 'rt': '22', 'des': 'Howard', 'pdist': 9511, 'dly': False, 'tatripid': '1040918', 'origtatripno'